In [1]:
from neuron import h, gui
import bokeh as bk
import bokeh.plotting as bkp
import numpy as np
from bokeh.palettes import Dark2_5 as palette
import itertools 
import ipywidgets as widgets
from branched_cable import branched_cable, run_current_clamp, run_sim

from matplotlib import pyplot as plt
%matplotlib inline

colors = itertools.cycle(palette)  
bkp.output_notebook()

%load_ext autoreload
%autoreload 2

celsius = 34.0  

Loading BokehJS ...

In [21]:
# make ball and stick neuron
# attach ball and stick

##### edit these variables to change cable properties
soma_diam = 12
soma_L=12
dend_diam=1
dend_L=250
dend_Ra=150
Rm=2800
stim_loc=1
tstop = 200
########

# build cell with these properties
cell = branched_cable()
cell.make_cable(soma_diam,soma_L,Rm=Rm,Vrest=-55)
#cell.add_branch('cable',1,dend_L,dend_diam,Ra=dend_Ra,Rm=Rm)
cell.channel_control('cable',kdr=True,gkdrbar=.01,na=True,kin=True,h=True)
cell.recording_vecs([['cable',.5,'v']])
#cell.noise_source('cable',.5,g_e0=.0121, g_i0=.0573)
RC = bkp.figure(plot_height=400,plot_width=400)
for i, color in zip(range(10),colors):
    
    for t in list(np.linspace(2,250,num=2500)):
        if i==0:
            cell.add_IClamp('cable',.5,.05*np.random.uniform(),.1,delay=t,stimname=str(t))
        else:
            cell.stims[str(t)].amp = .05*np.random.uniform()
    cell.add_IClamp('cable',.2,.003,50,delay = 100)
    t,results = run_sim(cell,tstop = 250)
    #t, results = run_current_clamp(cell,'iclamp',[.05]*8)
    vArr =  results['cable:0.50:v']

    # edit below to plot
    
    #keys = list(results.record_vecs.keys())

    RC.line(t,vArr[:],color=color)
bkp.show(RC)

# check by running current clamp with magnitude = 0


# put a synapse with alpha function at variable location along dendrite

# make strength and time constant of synapse variable

# inject gaussian noice soure via current clamp on soma

In [7]:
# ball and stick 

##### edit these variables to change cable properties
soma_diam = 12
soma_L=12
dend_diam=1
dend_L=250
dend_Ra=180
Rm=2800
stim_loc=1
tstop = 200
########


# build cell with these properties
cell = branched_cable()
cell.make_cable(soma_diam,soma_L,Rm=Rm,Vrest=-55)
cell.add_branch('cable',1,dend_L,dend_diam,Ra=dend_Ra,Rm=Rm)
cell.channel_control('cable',kdr=True,na=True,kin=True,h=True)
cell.channel_control('branch')#,kdr=True) #,h=True,kin=True)

cell.recording_vecs([['cable',.5,'v']])#,['branch',.25,'v'],['branch',.5,'v'],['branch',.75,'v']])
RC = bkp.figure(plot_height=400,plot_width=400)
for i, color in zip(range(1),colors):
    for t in list(np.linspace(2,250,num=2500)):
        if i==0:
            cell.add_IClamp('cable',.5,.01*np.random.normal(),.1,delay=t,stimname=str(t))
            #cell.add_IClamp('branch',.5,.01*np.random.normal(),.1,delay=t,stimname="branch"+str(t))
        else:
            cell.stims[str(t)].amp = .01*np.random.normal()
            #cell.stims["branch"+str(t)].amp = .01*np.random.normal()
    if i ==0:    
        #excitatory synapse
        stim = h.NetStim() # Make a new stimulator
        syn_ = h.ExpSyn(cell.segments['branch'](.5))
        syn_.e = 0
        syn_.tau = 1
        stim.number = 5
        stim.interval=.1
        stim.start = 80
        ncstim = h.NetCon(stim, syn_)
        ncstim.delay = 1
        ncstim.weight[0] = .04 # NetCon weight is a vector.
        cell.stims['expsyn']=ncstim
        
        delay = .1
        # inhibitory synapse
        istim = h.NetStim()
        isyn_ = h.ExpSyn(cell.segments['branch'](.4))
        isyn_.e = -80
        isyn_.tau = 1
        istim.number = 5
        istim.interval=.1
        istim.start = 80+delay
        incstim = h.NetCon(istim, isyn_)
        incstim.delay = 1
        incstim.weight[0] = .04 # NetCon weight is a vector.
        cell.stims['isyn']=incstim
        
    
        #cell.add_stim('cable',.5)
    t,results = run_sim(cell,tstop = 250)
    vArr =  results['cable:0.50:v']
    RC.line(t,vArr[:],color=color)
bkp.show(RC)


# add spine

# put alpha synapse on spine

# put inhibitory synapse on shaft

#active conductances in soma

# Ih in dendrites
    # change Ih reversal

In [31]:
# ball and stick 

##### edit these variables to change cable properties
soma_diam = 12
soma_L=12
dend_diam=1
dend_L=250
dend_Ra=180
Rm=2800
stim_loc=1
tstop = 200
########


# build cell with these properties
cell = branched_cable()
cell.make_cable(soma_diam,soma_L,Rm=Rm,Vrest=-55)
cell.add_branch('cable',1,dend_L,dend_diam,Ra=dend_Ra,Rm=Rm)
cell.add_branch('branch',.67,.01,.01,Ra=500,Rm=Rm,segname='spine',nseg=1)

cell.channel_control('cable',kdr=True,na=True,kin=True,h=True)
cell.channel_control('branch',kdr=True,h=True,kin=True)

cell.recording_vecs([['cable',.5,'v'],['branch',.25,'v'],
                    ['branch',.5,'v'],['branch',.75,'v']])

RC = bkp.figure(plot_height=400,plot_width=400)
for i, color in zip(range(1),colors):
    for t in list(np.linspace(2,250,num=2500)):
        if i==0:
            cell.add_IClamp('cable',.5,.05*np.random.normal(),.1,delay=t,stimname=str(t))
            #cell.add_IClamp('branch',.5,.01*np.random.normal(),.1,delay=t,stimname="branch"+str(t))
        else:
            cell.stims[str(t)].amp = .05*np.random.normal()
            #cell.stims["branch"+str(t)].amp = .01*np.random.normal()
    if i ==0:    
        #excitatory synapse
        stim = h.NetStim() # Make a new stimulator
        syn_ = h.ExpSyn(cell.segments['spine'](1))
        syn_.e = 100
        syn_.tau = 2
        stim.number = 10
        stim.interval=.1
        stim.start = 80
        ncstim = h.NetCon(stim, syn_)
        ncstim.delay = 1
        ncstim.weight[0] = .4 # NetCon weight is a vector.
        cell.stims['expsyn']=ncstim
        
        delay = 1
        # inhibitory synapse
        istim = h.NetStim()
        isyn_ = h.ExpSyn(cell.segments['branch'](.4))
        isyn_.e = -80
        isyn_.tau = 1
        istim.number = 5
        istim.interval=.1
        istim.start = 80+delay
        incstim = h.NetCon(istim, isyn_)
        incstim.delay = 1
        incstim.weight[0] = .04 # NetCon weight is a vector.
        cell.stims['isyn']=incstim
        
    
        #cell.add_stim('cable',.5)
    t,results = run_sim(cell,tstop = 250)
    vArr =  results['cable:0.50:v']
    RC.line(t,vArr[:],color=color)
bkp.show(RC)


# add spine

# put alpha synapse on spine

# put inhibitory synapse on shaft

#active conductances in soma

# Ih in dendrites
    # change Ih reversal

# add spine

# put alpha synapse on spine

# put inhibitory synapse on shaft

#active conductances in soma

# Ih in dendrites
    # change Ih reversal